In [2]:
import pandas as pd
import html
from src.data_preprocessing.cleaning import remove_tags

In [3]:
data = pd.read_csv("data/raw/bncc_first_classifier.csv", encoding="utf-8")

In [4]:
data_clone = data.copy()
# decoding from html
data_clone["question_c1"] = data["question"].astype("str").apply(html.unescape)
# removing strings
data_clone["question_c2"] = data_clone["question_c1"].apply(remove_tags)
# change columns names
data_clone.columns = ['id', 'question', 'bulletType', 'topic', 'slug', 'curricular_component', 'slug.1',
       'school_step', 'question_c1', 'question_c2']
# selecting specific columns
data_clone = data_clone[['question', 'bulletType', 'topic', 'curricular_component', 'school_step', 'question_c1', 'question_c2']]

Primeiro filtro que queremos aplicar aqui é o do nosso target de interesse, a área de conhecimento (`school_step`). Queremos apenas:

- Fundamental I
- Fundamental II
- Ensino Médio

In [5]:
data_clone["school_step"].value_counts()

Médio & Pré-Vestibular    35676
Fundamental II            33790
Fundamental I             18379
Concurso                   9875
Militar                    2200
OAB                          80
Name: school_step, dtype: int64

Como vimos, temos outras áreas de conhecimento que não se faz necessário para esse primeiro classificador. Portanto, precisamos dropar essas observações.

In [6]:
data_clone_correct_targets = data_clone[data_clone["school_step"].isin(["Médio & Pré-Vestibular", "Fundamental II", "Fundamental I"])]
data_clone_correct_targets

,question,bulletType,topic,curricular_component,school_step,question_c1,question_c2
0,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,Arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...
1,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,Matemática,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ..."
2,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,Espanhol,Fundamental II,<p>2. Complete os espaços abaixo com os artigo...,2. Complete os espaços abaixo com os artigos d...
4,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,Química,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...
5,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,4,Proposta,Redação,Fundamental I,<p><strong>Anexe nesse espaço uma foto da sua ...,Anexe nesse espaço uma foto da sua ilustração.
...,...,...,...,...,...,...,...
99994,<p>Leia a reportagem e responda a quest&atilde...,2,Outros,Língua Portuguesa,Fundamental I,<p>Leia a reportagem e responda a questão:</p>...,Leia a reportagem e responda a questão:Sobre a...
99995,"<p>Para responder à questão, leia o trecho do ...",2,Outros,Língua Portuguesa,Médio & Pré-Vestibular,"<p>Para responder à questão, leia o trecho do ...","Para responder à questão, leia o trecho do con..."
99996,"<h1 style=""list-style:none;margin:0px 0px 20px...",2,Práticas Corporais de Aventura: Natureza: Moun...,Educação Física,Fundamental II,"<h1 style=""list-style:none;margin:0px 0px 20px...",Em que pais surgiu o skate?
99998,<p>Os machos e fêmeas de mosquitos do gênero <...,2,Vida e evolução: Diversidade de ecossistemas: ...,Ciências,Fundamental II,<p>Os machos e fêmeas de mosquitos do gênero <...,Os machos e fêmeas de mosquitos do gênero Aede...


Outro ponto a se notar é que nós iremos, nesse primeiro momento, fazer a classificação da etapa escolar, e não iremos utilizar o componente curricular como preditor. Isso por que esse preditor aqui é também um código da BNCC e teremos uma estratégia só pra esse segundo classificador.

In [7]:
data_clone_sem_curr_comp = data_clone_correct_targets.drop(["curricular_component"], axis=1)
data_clone_sem_curr_comp

,question,bulletType,topic,school_step,question_c1,question_c2
0,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...
1,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ..."
2,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,Fundamental II,<p>2. Complete os espaços abaixo com os artigo...,2. Complete os espaços abaixo com os artigos d...
4,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...
5,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,4,Proposta,Fundamental I,<p><strong>Anexe nesse espaço uma foto da sua ...,Anexe nesse espaço uma foto da sua ilustração.
...,...,...,...,...,...,...
99994,<p>Leia a reportagem e responda a quest&atilde...,2,Outros,Fundamental I,<p>Leia a reportagem e responda a questão:</p>...,Leia a reportagem e responda a questão:Sobre a...
99995,"<p>Para responder à questão, leia o trecho do ...",2,Outros,Médio & Pré-Vestibular,"<p>Para responder à questão, leia o trecho do ...","Para responder à questão, leia o trecho do con..."
99996,"<h1 style=""list-style:none;margin:0px 0px 20px...",2,Práticas Corporais de Aventura: Natureza: Moun...,Fundamental II,"<h1 style=""list-style:none;margin:0px 0px 20px...",Em que pais surgiu o skate?
99998,<p>Os machos e fêmeas de mosquitos do gênero <...,2,Vida e evolução: Diversidade de ecossistemas: ...,Fundamental II,<p>Os machos e fêmeas de mosquitos do gênero <...,Os machos e fêmeas de mosquitos do gênero Aede...


In [8]:
data_clone_sem_curr_comp["bulletType"].value_counts()

2    68604
4    13881
1     2968
3     1179
5      999
7      209
6        5
Name: bulletType, dtype: int64

A variável bulletType contém items dos tipos:

1. Somatória
2. Múltipla Escolha
3. Certo e Errado
4. Discursiva
5. Redação ou items que é necessário enviar foto da resolução
6. Questões com mais de uma alternativa certa
7. Não representa nada em específico

Dessa forma, como queremos ao final classificar items, iremos remover questões relacionadas à redação, assim como questões do tipo 6.


In [9]:
data_clone_correct_bullets = data_clone_sem_curr_comp[~data_clone_sem_curr_comp["bulletType"].isin([5, 6])]
data_clone_correct_bullets

,question,bulletType,topic,school_step,question_c1,question_c2
0,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...
1,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ..."
2,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,Fundamental II,<p>2. Complete os espaços abaixo com os artigo...,2. Complete os espaços abaixo com os artigos d...
4,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...
5,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,4,Proposta,Fundamental I,<p><strong>Anexe nesse espaço uma foto da sua ...,Anexe nesse espaço uma foto da sua ilustração.
...,...,...,...,...,...,...
99994,<p>Leia a reportagem e responda a quest&atilde...,2,Outros,Fundamental I,<p>Leia a reportagem e responda a questão:</p>...,Leia a reportagem e responda a questão:Sobre a...
99995,"<p>Para responder à questão, leia o trecho do ...",2,Outros,Médio & Pré-Vestibular,"<p>Para responder à questão, leia o trecho do ...","Para responder à questão, leia o trecho do con..."
99996,"<h1 style=""list-style:none;margin:0px 0px 20px...",2,Práticas Corporais de Aventura: Natureza: Moun...,Fundamental II,"<h1 style=""list-style:none;margin:0px 0px 20px...",Em que pais surgiu o skate?
99998,<p>Os machos e fêmeas de mosquitos do gênero <...,2,Vida e evolução: Diversidade de ecossistemas: ...,Fundamental II,<p>Os machos e fêmeas de mosquitos do gênero <...,Os machos e fêmeas de mosquitos do gênero Aede...


In [12]:
# cleaning columns to use
df_to_eda = data_clone_correct_bullets.drop(["question", "question_c1"], axis=1)[["question_c2", "bulletType", "topic", "school_step"]].reset_index(drop=True)


In [14]:
df_to_eda.to_csv(path_or_buf="data/curated/df_to_eda.csv", index=False)